## Constrained Optimization

Suppose you want to maximize utility

$$u(x_1,x_2)=x_1*x_2$$

subject to a budget constraint

$$p_1*x_1+p_2*x_2\leq m$$

as well as the non-negativity constraints

$$x_1,x_2\geq 0.$$

There are several ways of doing so and neither looks very beginner friendly but as this is a common poblem in economics, I want to show you nevertheless how it can be done.

The first step is to rewrite the problem in a the computer science notation

$$u(x,p)=x[1]*x[2]$$

and writing the constraints as double inequalities solved for zero

$$-\infty\leq p[1]*x[1]+p[2]*x[2]-p[3]\leq 0$$

where $p$ is now a vector containing the parameters of the problem which are here the two prices and the budget, i.e. $p[1]$ is the price of good 1, $p[2]$ the price of good 2 and $p[3]$ is the budget. For the code below it is important to write everything as double inequalities and where we have no lower/upper restriction we do this by stating that a term has to be larger/less than (-)infinity. 

Note that the utility function does not depend on any of these parameters here (although it could in principle) but nevertheless the parameters have to be an argument in the $u$ function if you want your code to run without error. 

As our optimization routine does minimization instead of maximization, we define the negative of the utility function as our objective:

$$f(x,p)=-x[1]*x[2]$$

Finally, we need to add the non-negativity constraints:
$$0\leq x[1] \leq \infty$$
$$0\leq x[2] \leq \infty$$

If we wrote our constraints in vector notation it would look like this:

$$ \begin{bmatrix}
           -\infty\\
           0\\
           0
         \end{bmatrix}\leq  \begin{bmatrix}
           p[1]*x[1]+p[2]*x[2]-p[3]\\
           x[1]\\
           x[2]
         \end{bmatrix}\leq \begin{bmatrix}
           0\\
           \infty\\
           \infty
         \end{bmatrix}$$

We are almost there. However, the packages we are going to use want us to write all the constraints into one function and do this in a slightly odd format: We define a function $cons!(res,x,p)$ which has as a first argument ("res")  a vector that will be set inside the function to the value of the three constraints. That is,

In [1]:
function cons!(res,x,p)
    res[1]=p[1]*x[1]+p[2]*x[2]-p[3] #budget constraints
    res[2]=x[1] #non-negativity constraint for good 1
    res[3]=x[2] #non-negativity constraint for good 2
end

#lower bounds for the constraints
lcons = [-Inf,0.0,0.0]

#upper bounds for the constraints
ucons = [0.0,Inf,Inf]

3-element Vector{Float64}:
  0.0
 Inf
 Inf

In [2]:
#loading several optimization packages
using Optimization, OptimizationMOI, OptimizationOptimJL, Ipopt
using ForwardDiff, ModelingToolkit

#defining the objective
f(x,p)=-x[1]*x[2]

#set starting value for x
start = [1.0,1.0]

#set parameter values, here: p1=3, p2=5, m=20
parameters = [3.0,5.0,20.0]

#defining the optimization problem
optprob = OptimizationFunction(f,Optimization.AutoForwardDiff(),cons=cons!)

#defining the problem with concrete starting value parameters and bounds for the constraints
prob = OptimizationProblem(optprob,start,parameters,lcons=lcons,ucons=ucons)

# solve the problem using the algorithm IPNewton
sol = solve(prob,IPNewton())

retcode: Success
u: 2-element Vector{Float64}:
 3.333333333333333
 2.0000000000000004

"retcode: Success" tells us that the optimization algorithm has successfully converged, i.e. it found a minimum. The Vector that is then returned is the minimizer.

### Equality constraints

Above we had a problem with inequality constraints but it is easy to adapt it to using equality constraints. Suppose you wanted to solve the same problem as above but you wanted to use the budget constraint as an equality, i.e. 
$$p_1 x_1+p_2 x_2=m.$$
The trick is that you can write an equation as two inequalities. In our case, the equation above is equivalent to 
$$0\leq p_1 x_1+p_2 x_2-m\leq 0.$$
And in this form we can use feed the equation as tow inequalities into julia. That is, the only difference to the code above is that we would now use:

In [ ]:
#lower bounds for the constraints
lcons = [0.0,0.0,0.0]  #instead of lcons = [-Inf,0.0,0.0]

#upper bounds for the constraints
ucons = [0.0,Inf,Inf]

You may find it strange to provide an equality constraint as two inequalities but note that this gives us a lot of flexibility. In particular, we can provide a mix of equality and inequality constraint to Julia as we just did where the non-negativity constraints are inequality constraints while the budget constraint was an equality constraint.